In [1]:
# Libraries for selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
# Waited conditionals for Selenium
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Data Manipulations libraries
import pandas as pd

# Regular expressions library
import re

# Datetime library
import time

In [2]:
# Open the web browser
driver = webdriver.Edge()
# Maximize the window
driver.maximize_window()

In [3]:
# Acceder al SIGMAA
url = 'https://uclb.ucaribe.edu.mx/sigmaav2/'
driver.get(url)

In [4]:
# Biblioteca para leer archivos
import os
# Biblioteca para leer .env
from dotenv import load_dotenv
# Cargamos variables de ambiente
load_dotenv()

# Obtener las credenciales del estudiante
id_user = os.getenv('ID')
password = os.getenv('PASS')
MYSQL_ADDON_URI = os.getenv('MYSQL_ADDON_URI')

In [5]:
# Buscar el campo de usuario y escribir el username
userinput = driver.find_element(By.XPATH, '/html/body/div[2]/form/div/span[2]/input')
userinput.send_keys(id_user)
# Buscar el campo de contraseña y escribir el password
passinput = driver.find_element(By.XPATH, '/html/body/div[2]/form/div/input')
passinput.send_keys(password)
# Buscar el botón de submit y dar clic para iniciar sesión
submitinput = driver.find_element(By.XPATH, '/html/body/div[2]/form/button')
submitinput.click()

In [6]:
sistemaPagos = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div/div/ul[2]/li[7]/a')
sistemaPagos.click()

In [7]:
# Extraer la tabla usando webscrapping
dfs = pd.read_html(driver.page_source)

In [8]:
pagos = dfs[0].drop(['Opciones', 'Opciones.1'], axis = 1)

In [9]:
pagos

,No.,Id. Adeudo,Periodo,Fecha,Concepto,Monto,Vencimiento,Estatus
0,1,239213,202201,2022-01-07,337 / Convenio No. 4680 pago 1/4,567.5,2022-01-07,Pagada
1,3,239214,202201,2022-01-07,337 / Convenio No. 4680 pago 2/4,567.5,2022-02-09,Pagada
2,4,239215,202201,2022-01-07,337 / Convenio No. 4680 pago 3/4,567.5,2022-03-09,Pagada
3,5,239216,202201,2022-01-07,337 / Convenio No. 4680 pago 4/4,567.5,2022-04-09,Pagada


In [10]:
# Importar biblioteca para conexion a base de datos
from sqlalchemy import create_engine
# Establecer conexion a base de datos
engine = create_engine(MYSQL_ADDON_URI+"?ssl=true")

In [11]:
nombre_tabla = "pagos_" + id_user
pagos.to_sql(nombre_tabla, con = engine, if_exists='replace')

4

In [12]:
# Cerrar sesión
logout = driver.find_element(By.XPATH, '/html/body/div[2]/div/form/a')
logout.click()
# Cerrar el navegador
driver.quit()